In [12]:
import spacy
import re
from spacy.tokens import Doc
import json
goalpos = json.load(open("sequoia.test.json", "r"))


# Charge the french model from spacy
nlp = spacy.load("fr_core_news_sm")

#function given in the exercice by the teacher   
def predict_pos(sentence, model):
    model.tokenizer = lambda x: Doc(model.vocab, x.split())
    return [token.pos_ for token in model(sentence)]
    
#print(predict_pos("J' aime le chocolat", nlp))
""" 
1. What is lambda function in python and why do we use such a function in line 4 ? 
    Réponse: Lambda is an anonyme function that turns the sentence X into tokens with split function on spaces (" ")
    it returns a DOC object with the tokenized sentence. It avoids using a more complexe tokenizer this one onlys split on spaces
2. Explain the last line of the predict_pos function.
    Réponse: this part:" model(sentence)" calls the simplified tokenizer with lambda function on sentence and it returns a DOc object with 
    the tokenized sentence
    this part:"for token in model(sentence)" boucle on each word in the sentence
    this part"token.pos_ "associates a tag to the each token
    thepredict_pos  returns for the setence "J' aime le chocolat this list:['PRON', 'VERB', 'DET', 'NOUN']
3. Write a function, that takes as input a list of sentences and returns a list of PoS annotations
 (i.e. a list of lists of PoS)
 """
def predict_pos_list(list_sentence, model):
    # cut the minu corous on., ?, !, ou ...
    sentences = re.split(r'[.!?]+', list_sentence)
    # clean the sentences
    sentences = [s.strip() for s in sentences if s.strip() != ""]
    list_pos=[]# to store list of lists of PoS
    for l in sentences:
        list_pos.append(predict_pos(l, model))# calls predict_pos on a single sentence to return the list of POS
    return list_pos# return list of lists of PoS

        
#print(predict_pos_list("J' aime le chocolat. Je n'aime pas le lait", nlp))

"""
 4. Whydowe(have to) consider tokenized sentences to evaluate a PoS tagger?
 because the POS tagger gives words' labels not sentences'labels for example usually a  sentence is not a DET except one word sentences.
 5. Implement each of the evaluation metric defined in this section."""
"""
Create a dico where pos labels are keys and words value
sentence_accuracy create a vector if 1 value of the vector is Zero it returns 0 or if the length is different
micro_word_accuracy create a vector and calculate sum(vector)/ number of words
macro_word_accuracy creates a dictionnary with postags and their occurences in the list of poses only if the vecto indicates it is accurate
"""
"""refaire toutes les fonctions pour que dico soit de la forme{'tokens': ['Je', "n'", 'aime', 'pas', 'le', 'lait'], 'pos': ['PRON', 'NEG', 'VERB', 'ADV', 'DET', 'NOUN']}
{'tokens': ['Tu', 'adores', 'le', 'chocolat'], 'pos': ['PRON', 'VERB', 'DET', 'NOUN']}"""
def sentence_accuracy(poslist,dico):
    vector=[]
    accuracy=0
    lengthposlit=len(poslist)
    goldpos=dico.get("pos")
    lengthgoldpos=len(goldpos)
    minlength=min(lengthposlit, lengthgoldpos)
    compteur=0
    while compteur<minlength:
            if poslist[compteur]==goldpos[compteur]:
                vector.append(1)
                compteur=compteur+1
            else:
                vector.append(0)
                compteur=compteur+1
    if lengthposlit>lengthgoldpos:# adds 0 to the vector if poslit longueur than dico
        for i  in range(compteur,lengthposlit):
            vector.append(0)
    if sum(vector)==lengthgoldpos:
        accuracy=1
    else:
        accuracy=0
    return vector, accuracy
    
def micro_word_accuracy(poslist,dico):
    vector= sentence_accuracy(poslist,dico)[0]
    word_accuracy=sum(vector)

    return int(word_accuracy)
def macro_word_accuracy(poslist,dico,dico_poslabels):# has alreday  dico_poslabels as an argyment to enrich it
    vector= sentence_accuracy(poslist,dico)[0]
    goldpos=dico.get("pos")
    lengthgoldpos=len(goldpos)
    lenvector=len(vector)
    lenmin=min(lenvector,lengthgoldpos)
    compteur=0
    while compteur<lenmin:
        if goldpos[compteur] in dico_poslabels and vector[compteur]==1:#if alreaddy in dico that adds 1
                dico_poslabels[goldpos[compteur]]=dico_poslabels[goldpos[compteur]]+1
        if goldpos[compteur] not in dico_poslabels and vector[compteur]==1:#if not in dico creates a key
            dico_poslabels[goldpos[compteur]]=1
        if goldpos[compteur] not in dico_poslabels and vector[compteur]==0:#if not in dico creates a key creates a key with value 0
            dico_poslabels[goldpos[compteur]]=0             
        compteur=compteur+1
    #add pos labels that have not been predicted at all
    for compteur  in range(compteur,lengthgoldpos):
        if goldpos[compteur] not in dico_poslabels:
            dico_poslabels[goldpos[compteur]]=0  
    
    return dico_poslabels
        

def postagger_accuracy(poslist_list, dico):
    sentence=0
    proportion=0
    lengthtexte=0
    correctpos={}
    
    for i in range(len(poslist_list)):
        sentence=sentence+sentence_accuracy(poslist_list[i],dico[i])[1]
        proportion= proportion+micro_word_accuracy(poslist_list[i],dico[i])
        goldpos=dico[i].get("pos")
        lengthgoldpos=len(goldpos)
        lengthtexte=lengthtexte+lengthgoldpos
        correctpos=macro_word_accuracy(poslist_list[i],dico[i],correctpos)#update the correctpos by adding new keys if necessary or updating the key value
    proportion=(proportion/lengthtexte)*100
    return sentence,proportion,correctpos
        
        
"""6. How can these new metrics be computed easily (i.e. by using the functions you have
 already implemented)
 They can be easily implemented because they coorrespond to the number of sentences in the text minus sentence_accuracy[1] or the numbers of words in the text minus
 micro_word_accuracy
"""
def word_error_rate(poslist_list, dico):
    word_accurate=0
    lengthtexte=0
    for i in range(len(poslist_list)):
        word_accurate=word_accurate+micro_word_accuracy(poslist_list[i],dico[i])
        goldpos=dico[i].get("pos")
        lengthgoldpos=len(goldpos)
        lengthtexte=lengthtexte+lengthgoldpos
    word_error_rate_value=((lengthtexte-word_accurate)/lengthtexte)*100
    return (int(word_error_rate_value))

    
#print(word_error_rate(lpos_texte, dictionnaires ))


def corpus_stats(filepath):
    """
    Affiche le nombre de phrases et de mots dans un corpus JSON
    (liste de dictionnaires avec clés 'tokens' et 'pos' ou 'goldpos').
    """
    with open(filepath, "r", encoding="utf-8") as f:
        corpus = json.load(f)

    # Nombre de lignes = nombre d'entrées dans la liste
    nb_lignes = len(corpus)
    nb_mots=0 # initialize varaible at 0
    listkey=[]
    i=0
    while i<len(corpus):# this whole loop must be changed
        for k in corpus[i]:
            listemots=corpus[i][k]
            nb_mots=nb_mots+len(listemots)# not efficient becuase count pos and it is useless but works
        i=i+1
    nb_mots=int(nb_mots/2) # divides by 2 to retrieve the Pos labels of the number of words
    
    print(f"Nombre de phrases : {nb_lignes}")
    print(f"Nombre total de mots : {nb_mots}")

    return nb_lignes, nb_mots
#print(corpus_stats("sequoia.test.json"))
""" 7. Howmanysentences and words are there in the Sequoia corpus
Nombre de phrases : 456
Nombre total de mots : 10354
(456, 10354)
8. Evaluate the performance achieved by the tagger described in Section 2 on the corpus.
 Interpret.
 The performance achieved by the tagger are very low but this is exepcted as we used a minimal and simplits tagger """
def corpus_tokens_only(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        corpus = json.load(f)
    liste_phrase=[]
    i=0
    while i<len(corpus):# this whole loop must be changed
        for k in corpus[i]:
            if k=="tokens":
                liste_phrase.append(" ".join(corpus[i][k]))
        i=i+1
    str_corpus="".join(liste_phrase)
    return str_corpus
       
                
corpusbis=corpus_tokens_only("sequoia.test.json")# list of tokenize sentences from the sequoia file
lpos=predict_pos_list(corpusbis, nlp)
correctmacro={}
#print(sentence_accuracy(['PRON', 'VERB', 'SCONJ', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'PRON', 'VERB', 'ADV', 'NUM', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'NOUN', 'ADP', 'DET', 'PROPN', 'PUNCT', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'SCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADP', 'DET', 'NOUN', 'NOUN', 'CCONJ', 'PUNCT', 'ADV', 'ADV', 'PUNCT', 'DET', 'NOUN', 'VERB', 'PUNCT', 'DET', 'NOUN', 'ADP', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN'],{'tokens': ['cela', 'signifie', 'que', 'leur', 'consommation', 'énergétique', ',', 'qui', 'représente', 'actuellement', '10', '%', 'de', 'la', 'consommation', 'énergétique', 'moyenne', 'de', "l'", 'UE', ',', 'enregistrera', 'une', 'forte', 'augmentation', 'à', 'mesure', "qu'", 'ils', 'exigeront', 'des', 'commodités', 'élémentaires', 'comme', "l'", 'eau', 'chaude', 'et', ',', 'peut-être', 'même', ',', "l'", 'air', 'conditionné', ',', 'des', 'moyens', 'de', 'transport', 'et', 'la', 'modernisation', 'de', 'leurs', 'industries', '.'], 'pos': ['PRON', 'VERB', 'SCONJ', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'PRON', 'VERB', 'ADV', 'NUM', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADJ', 'ADP', 'DET', 'PROPN', 'PUNCT', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'SCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADJ', 'CCONJ', 'PUNCT', 'ADV', 'ADV', 'PUNCT', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'DET', 'NOUN', 'ADP', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT']}))
print(word_error_rate(lpos, goalpos))
"""temps passé 13h"""

77


'temps passé 13h'